In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly
import plotly.plotly as py
from scipy import stats
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import sklearn.metrics
import statsmodels.api as sm
import statsmodels.formula.api as smf
from functools import reduce
from operator import and_
import operator
import time

C:\Users\Michael\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



# Initialization and definitions

In [2]:
dfs = {} # dict of data frames for each year
filtered_dfs = {} # dict of data frames with processed and filtered columns
model_dfs = {} # raw data for fit
model_filtered_dfs = {} # filtered data for fit

In [3]:
state_dict = { 1: 'AL',  2: 'AK',  4: 'AZ',  5: 'AR',  6: 'CA',  8: 'CO',  9: 'CT',
              10: 'DE', 12: 'FL', 13: 'GA', 15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN',
              19: 'IA', 20: 'KS', 21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD', 25: 'MA',
              26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV',
              33: 'NH', 34: 'NJ', 35: 'NM', 36: 'NY', 37: 'NC', 38: 'ND', 39: 'OH',
              40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC', 46: 'SD', 47: 'TN',
              48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI',
              56: 'WY'}
states = list(state_dict.values())

factors = ['NonDrinker', 'NonSmoker', 'NotOverweight', 'Exercises',
           'State', 'EnoughSleep', 'NoDepressive']
model_factors = ['NoPhys14D', 'NoMent14D', 'HasHealthPlan', 'MedCostOK',
                 'Checkup1Yr', 'Exercises', 'EnoughSleep', 'NeverHeartAttack',
                 'NeverHeartDisease', 'NeverStroke', 'NoAsthma',
                 'NoOtherCancer', 'NoCOPD', 'NoArthritis', 'NoDepressive',
                 'NoKidneyDisease', 'NoDiabetes', 'Dentist1Yr', 'NoTeethRemoved',
                 'AttendedCollege', 'Veteran', 'ChildrenInHousehold', 'NotOverweight',
                 'NotDeaf', 'NotBlind', 'DecisionOK', 'DressOK',
                 'WalkOK', 'AloneOK', 'NonSmoker', 'NonECig', 'NonDrinker',
                 'AlwaysSeatbelt', 'NoHIVRisk',
                 '_INCOMG',
                 'NoSkinCancer', 'FluShot1Yr', 'TestedHIV', 'HasDoctor'
                ]

value_filter = {'_STATE': state_dict.keys(),
                'GENHLTH': [1,2,3,4,5],
                '_PHYS14D': [1,2,3],
                '_MENT14D': [1,2,3],
                'HLTHPLN1': [1,2],
                'PERSDOC2': [1,2,3],
                'MEDCOST': [1,2],
                'CHECKUP1': [1,2,3,4],
                'EXERANY2': [1,2],
                'SLEPTIM1': range(1, 25),
                'CVDINFR4': [1,2],
                'CVDCRHD4': [1,2],
                'CVDSTRK3': [1,2],
                '_ASTHMS1': [1,2,3],
                'CHCSCNCR': [1,2],
                'CHCOCNCR': [1,2],
                'CHCCOPD1': [1,2],
                'HAVARTH3': [1,2],
                'ADDEPEV2': [1,2],
                'CHCKIDNY': [1,2],
                'DIABETE3': [1,2,3,4],
                '_DENVST2': [1,2],
                'RMVTETH3': [1,2,3,8],
                'EDUCA': [1,2,3,4,5,6],
                'VETERAN3': [1,2],
                'CHILDREN': range(1, 89),
                '_INCOMG': [1,2,3,4,5],
                '_RFBMI5': [1,2],
                '_BMI5CAT': [1,2,3,4],
                'DEAF': [1,2],
                'BLIND': [1,2],
                'DECIDE': [1,2],
                'DIFFWALK': [1,2],
                'DIFFDRES': [1,2],
                'DIFFALON': [1,2],
                '_RFSMOK3': [1,2],
                'ECIGARET': [1,2],
                '_CURECIG': [1,2],
                'DRNKANY5': [1,2],
                'FLUSHOT6': [1,2],
                'SEATBELT': [1,2,3,4,5,8],
                'BLDSTOOL': [1,2,0],
                'HADSIGM3': [1,2,0],
                'HIVTST6': [1,2],
                'HIVRISK4': [1,2]
               }

f_convert = {'State': ['_STATE', state_dict],
             'VeryGoodHealth': ['GENHLTH', {1: True, 2: True, 3: False, 4: False, 5: False}],
             'GoodHealth': ['GENHLTH', {1: True, 2: True, 3: True, 4: False, 5: False}],
             'NoPhys14D': ['_PHYS14D', {1: True, 2: False, 3: False}],
             'NoMent14D': ['_MENT14D', {1: True, 2: False, 3: False}],
             'HasHealthPlan': ['HLTHPLN1', {1: True, 2: False}],
             'HasDoctor': ['PERSDOC2', {1: True, 2: True, 3: False}],
             'MedCostOK': ['MEDCOST', {2: True, 1: False}],
             'Checkup1Yr': ['CHECKUP1', {1: True, 2: False, 3: False, 4: False}],
             'Exercises': ['EXERANY2', {1: True, 2: False}],
             'EnoughSleep': ['SLEPTIM1', {h: (h >= 8) for h in range(1, 25)}],
             'NeverHeartAttack': ['CVDINFR4', {2: True, 1: False}],
             'NeverHeartDisease': ['CVDCRHD4', {2: True, 1: False}],
             'NeverStroke': ['CVDSTRK3', {2: True, 1: False}],
             'NoAsthma': ['_ASTHMS1', {2: True, 3: True, 1: False}],
             'NoSkinCancer': ['CHCSCNCR', {2: True, 1: False}],
             'NoOtherCancer': ['CHCOCNCR', {2: True, 1: False}],
             'NoCOPD': ['CHCCOPD1', {2: True, 1: False}],
             'NoArthritis': ['HAVARTH3', {2: True, 1: False}],
             'NoDepressive': ['ADDEPEV2', {2: True, 1: False}],
             'NoKidneyDisease': ['CHCKIDNY', {2: True, 1: False}],
             'NoDiabetes': ['DIABETE3', {2: True, 3: True, 4: True, 1: False}],
             'Dentist1Yr': ['_DENVST2', {1: True, 2: False}],
             'NoTeethRemoved': ['RMVTETH3', {8: True, 1: False, 2: False, 3: False}],
             'AttendedCollege': ['EDUCA', {5: True, 6: True, 1: False, 2: False, 3: False, 4: False}],
             'Veteran': ['VETERAN3', {1: True, 2: False}],
             'ChildrenInHousehold': ['CHILDREN', {c: (c != 88) for c in range(1, 89)}],
             'NotOverweight': ['_RFBMI5', {1: True, 2: False}],
             'NotUnderweight': ['_BMI5CAT', {2: True, 1: False}],
             'NotDeaf': ['DEAF', {2: True, 1: False}],
             'NotBlind': ['BLIND', {2: True, 1: False}],
             'DecisionOK': ['DECIDE', {2: True, 1: False}],
             'DressOK': ['DIFFDRES', {2: True, 1: False}],
             'WalkOK': ['DIFFWALK', {2: True, 1: False}],
             'AloneOK': ['DIFFALON', {2: True, 1: False}],
             'NonSmoker': ['_RFSMOK3', {1: True, 2: False}],
             'NonECig': ['_CURECIG', {1: True, 2: False}],
             'NonDrinker': ['DRNKANY5', {2: True, 1: False}],
             'FluShot1Yr': ['FLUSHOT6', {1: True, 2: False}],
             'AlwaysSeatbelt': ['SEATBELT', {1: True, 8: True, 2: False, 3: False, 4: False, 5: False}],
             'TestedHIV': ['HIVTST6', {1: True, 2: False}],
             'NoHIVRisk': ['HIVRISK4', {2: True, 1: False}]
            }

f_label = {'NotOverweight': 'Not overweight',
           'NonSmoker': 'Non-smoker',
           'NoDepressive': 'No depressive disorder',
           'Exercises': 'Exercises regularly',
           'EnoughSleep': '8+ hours of sleep'
          }

In [4]:
def load_data(year):
    print('Loading data for year ' + str(year))
    layout_fname = 'C:/Users/Michael/Documents/data incubator application/brfss' + str(year) + '_layout.csv'
    data_fname = 'C:/Users/Michael/Documents/data incubator application/LLCP' + str(year) + '.ASC'
    layout = pd.read_csv(layout_fname, names=['StartCol', 'Name', 'Length'])
    keep_columns = (['GENHLTH'] +
                    [f_convert[f][0] for f in factors if f in f_convert.keys()] +
                    [f for f in factors if f not in f_convert.keys()])
    layout_filtered = layout[layout['Name'].isin(keep_columns)]
    colspecs = list(zip(layout_filtered['StartCol'] - 1,
                        layout_filtered['StartCol'] + layout_filtered['Length'] - 1))
    df = pd.read_fwf(data_fname, colspecs=colspecs,
                     header=None, names=layout_filtered['Name'])
#     df = df[~df['_STATE'].isin([11, 66, 72, 78])] # Drop DC, Guam, Puerto Rico, Virgin Islands
#     df['State'] = df['_STATE'].replace(state_dict)
    print('Finished loading data for year ' + str(year))
    return df

def load_data_model(year):
    # Load data with more columns for use in model fitting.
    print('Loading data for year ' + str(year))
    layout_fname = 'C:/Users/Michael/Documents/data incubator application/brfss' + str(year) + '_layout.csv'
    data_fname = 'C:/Users/Michael/Documents/data incubator application/LLCP' + str(year) + '.ASC'
    layout = pd.read_csv(layout_fname, names=['StartCol', 'Name', 'Length'])
    keep_columns = (['GENHLTH', '_STATE'] +
                    [f_convert[f][0] for f in model_factors if f in f_convert.keys()] +
                    [f for f in model_factors if f not in f_convert.keys()])
    layout_filtered = layout[layout['Name'].isin(keep_columns)]
    colspecs = list(zip(layout_filtered['StartCol'] - 1,
                        layout_filtered['StartCol'] + layout_filtered['Length'] - 1))
    df = pd.read_fwf(data_fname, colspecs=colspecs,
                     header=None, names=layout_filtered['Name'])
    print('Finished loading data for year ' + str(year))
    return df

def filter_for_map(year, factor):
    final_columns = ['GENHLTH', 'State', factor]
    keep_columns = []
    
    # Identify raw and processed final columns
    for col in final_columns:
        if col in dfs[year].columns:
            keep_columns += [col]
        else:
            keep_columns += [f_convert[col][0]]
    df = dfs[year][keep_columns]
    
    # Remove unwanted values
    df = df[reduce(and_, [df[col].isin(value_filter[col]) for col in df.columns])]
    
    # Create processed columns
    for col in final_columns:
        if col not in dfs[year].columns:
            df[col] = df[f_convert[col][0]].map(f_convert[col][1])
            
    return df

def filter_for_bar(years, factors, group=None):
    final_columns = ['GENHLTH'] + factors
    if group is not None:
        final_columns += [group]
    keep_columns = []
    dfs_filtered = {}
    
    for i, year in enumerate(years):
        
        # Identify raw and processed final columns
        if i == 0:
            for col in final_columns:
                if col in dfs[year].columns:
                    keep_columns += [col]
                else:
                    keep_columns += [f_convert[col][0]]

        df = dfs[year][keep_columns]

        # Remove unwanted values
        df = df[reduce(and_, [df[col].isin(value_filter[col]) for col in df.columns])]

        # Create processed columns
        for col in final_columns:
            if col not in dfs[year].columns:
                df[col] = df[f_convert[col][0]].map(f_convert[col][1])
        
        dfs_filtered[year] = df

    return dfs_filtered

def filter_for_model(year, group=None):
    final_columns = ['GENHLTH', 'VeryGoodHealth', 'GoodHealth', 'State'] + model_factors
    if group is not None:
        final_columns += [group]
    keep_columns = []
    
    # Identify raw and processed final columns
    for col in final_columns:
        if col in model_dfs[year].columns:
            if col not in keep_columns:
                keep_columns += [col]
        else:
            if f_convert[col][0] not in keep_columns:
                keep_columns += [f_convert[col][0]]

    df = model_dfs[year][keep_columns]

    # Remove unwanted values
    print('Before filtering: df length is ' + str(len(df)))
    df = df[reduce(and_, [df[col].isin(value_filter[col]) for col in df.columns])]
    print('After filtering: df length is ' + str(len(df)))

    # Create processed columns
    for col in final_columns:
        if col not in model_dfs[year].columns:
            df[col] = df[f_convert[col][0]].map(f_convert[col][1])

    return df

# Choropleth map

In [5]:
year = 2016
if year not in dfs.keys():
    dfs[year] = load_data(year)

Loading data for year 2016
Finished loading data for year 2016


In [6]:
factor = 'NotOverweight'
filtered_df = filter_for_map(2016, factor)
agg_true = filtered_df[['State', 'GENHLTH']][filtered_df[factor] == True].groupby('State').agg(np.mean)
agg_false = filtered_df[['State', 'GENHLTH']][filtered_df[factor] == False].groupby('State').agg(np.mean)
agg_diff = agg_false - agg_true

In [8]:
for col in agg_diff.columns:
    agg_diff[col] = agg_diff[col].astype(str)

scl = [[0.0, 'rgb(180,230,180)'], [1.0, 'rgb(80,150,80)']]

agg_diff['text'] = agg_diff.index + '<br>' + 'Health score difference '+ agg_diff['GENHLTH']

data = [dict(type='choropleth',
             colorscale = scl,
             autocolorscale = False,
             locations = agg_diff.index,
             z = agg_diff['GENHLTH'].astype(float),
             locationmode = 'USA-states',
             text = agg_diff['text'],
             marker = dict(line = dict (color = 'rgb(255,255,255)', width = 2)),
             colorbar = dict(title = 'Difference in<br>mean health score'))]

layout = dict(title = ('Difference in Self-Reported General Health<br>' +
                       'for overweight and non-overweight individuals by State'),
              geo = dict(scope='usa',
                         projection=dict( type='albers usa' ),
                         showlakes = True,
                         lakecolor = 'rgb(255, 255, 255)'))
    
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='overweight-map')

Static copy of the image:

<img src='overweight-map.png'>

In [9]:
factor = 'NoDepressive'
filtered_df = filter_for_map(2016, factor)
agg_true = filtered_df[['State', 'GENHLTH']][filtered_df[factor] == True].groupby('State').agg(np.mean)
agg_false = filtered_df[['State', 'GENHLTH']][filtered_df[factor] == False].groupby('State').agg(np.mean)
agg_diff = agg_false - agg_true

In [10]:
for col in agg_diff.columns:
    agg_diff[col] = agg_diff[col].astype(str)

scl = [[0.0, 'rgb(220,170,220)'], [1.0, 'rgb(130,70,130)']]

agg_diff['text'] = agg_diff.index + '<br>' + 'Health score difference '+ agg_diff['GENHLTH']

data = [dict(type='choropleth',
             colorscale = scl,
             autocolorscale = False,
             locations = agg_diff.index,
             z = agg_diff['GENHLTH'].astype(float),
             locationmode = 'USA-states',
             text = agg_diff['text'],
             marker = dict(line = dict (color = 'rgb(255,255,255)', width = 2)),
             colorbar = dict(title = 'Difference in<br>mean health score'))]

layout = dict(title = ('Difference in Self-Reported General Health<br>' +
                       'for individuals with and without a depressive disorder by State'),
              geo = dict(scope='usa',
                         projection=dict( type='albers usa' ),
                         showlakes = True,
                         lakecolor = 'rgb(255, 255, 255)'))
    
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='depressive-map')

Static copy of the image:

<img src='depressive-map.png'>

In [11]:
factor = 'Exercises'
filtered_df = filter_for_map(2016, factor)
agg_true = filtered_df[['State', 'GENHLTH']][filtered_df[factor] == True].groupby('State').agg(np.mean)
agg_false = filtered_df[['State', 'GENHLTH']][filtered_df[factor] == False].groupby('State').agg(np.mean)
agg_diff = agg_false - agg_true

In [12]:
for col in agg_diff.columns:
    agg_diff[col] = agg_diff[col].astype(str)

scl = [[0.0, 'rgb(180,180,230)'], [1.0, 'rgb(80,80,150)']]

agg_diff['text'] = agg_diff.index + '<br>' + 'Health score difference '+ agg_diff['GENHLTH']

data = [dict(type='choropleth',
             colorscale = scl,
             autocolorscale = False,
             locations = agg_diff.index,
             z = agg_diff['GENHLTH'].astype(float),
             locationmode = 'USA-states',
             text = agg_diff['text'],
             marker = dict(line = dict (color = 'rgb(255,255,255)', width = 2)),
             colorbar = dict(title = 'Difference in<br>mean health score'))]

layout = dict(title = ('Difference in Self-Reported General Health<br>' +
                       'for individuals with and without regular exercise by State'),
              geo = dict(scope='usa',
                         projection=dict( type='albers usa' ),
                         showlakes = True,
                         lakecolor = 'rgb(255, 255, 255)'))
    
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='exercise-map')

Static copy of the image:

<img src='exercise-map.png'>